In [1]:
we = int(input("Enter the number of Weeks to run the model"))

# Importing required Libraries

In [2]:
import pandas as pd
from pulp import *
from num2words import *

# Data Preprocessing

In [3]:
inventory0 = pd.read_csv('inventory.csv')
sales0 = pd.read_csv('Weekly-Sales.csv')
productdesc0 = pd.read_csv('products.csv').set_index('Product_ID')
stores0 = pd.read_csv('stores.csv')

In [4]:
# Keeping More weeks will require more computation time
week = [None]*(we+1)
for i in range(0,(we+1)):
    week[i] =i

In [5]:
stores = [None]*50
for i in range(0,50):
    stores[i] = i+1
products = [None]*35
for i in range(0,35):
    products[i]=i+1

In [6]:
sales = {}
for s in stores:
    for p in products:
        for w in week:
            sales[(s,p,w)]=(sales0[(sales0['Store ID']==s)&(sales0['Product ID']==p)&(sales0['Week']==w)&(sales0['Year of Date']==2017)]['Count of sales'])
            if len(sales[s,p,w]>0):
                sales[s,p,w]=int(sales[s,p,w])
            else:
                sales[s,p,w]=0
del sales0     

In [7]:
openinginventory = {}
for s in stores:
    for p in products:
        openinginventory[(s,p)]=inventory0[(inventory0['Store_ID']==s)&(inventory0['Product_ID']==p)]['Stock_On_Hand']
        if len(openinginventory[(s,p)]>0):
            openinginventory[(s,p)] = int(openinginventory[(s,p)])
        else:
            openinginventory[(s,p)] = 0
del inventory0

In [8]:
Pcost = {}
Pprice = {}
Pprofit = {}
for p in products:
    Pcost[p]=float(productdesc0.loc[p]['Product_Cost'][1:5])
    Pprice[p]=float(productdesc0.loc[p]['Product_Price'][1:5])
    Pprofit[p] = Pprice[p] - Pcost[p]


# Model

## Declaring Objective Variable

In [9]:
# Objective Variable
z = LpProblem('Maximize_the_profit',LpMaximize)

## Decision Variables

In [10]:
# Decision Variable
x = LpVariable.dict('Buying',(stores,products,week),lowBound=0,cat='Integer')
y = LpVariable.dict('Selling',(stores,products,week),lowBound=0,cat='Integer')
capital_unused = LpVariable.dict('Selling',(week),lowBound=0,cat='Continous')
inventory = LpVariable.dict('Inventory',(stores,products,week),lowBound=0,cat='Integer')

## Objective Function

In [11]:
# Objective Function
# We want maximum profit in each week 
# We are doing false sales in week zero so ......
z += lpSum(y[s,p,w]*Pprofit[p] for s in stores for p in products for w in week) - lpSum(y[s,p,0]*Pprofit[p] for s in stores for p in products)

## Constraints

### Constraints for Weeks 2 to 6

In [12]:
for w in range(2,(we+1)):
    # We Cannot Purchase more than Capital Available
    z += lpSum(x[s,p,w]*Pcost[p] for s in stores for p in products) + capital_unused[w] == lpSum(y[s,p,w-1]*Pprice[p] for s in stores for p in products) + capital_unused[w-1] 
for s in stores:
    for p in products:
        for w in range(2,(we+1)):
                z += inventory[s,p,w] == x[s,p,w]
                z += y[s,p,w] <= sales[s,p,w]
                z += y[s,p,w] <= inventory[s,p,w]

### Constraints for Week 1

In [13]:
for s in stores:
    for p in products:
        z += inventory[s,p,1] == openinginventory[s,p]
        z += y[s,p,1] <= sales[s,p,1]
        z += y[s,p,1] <= inventory[s,p,1]
        z += capital_unused[1] == 0

## Running our model

In [14]:
status = z.solve()
total_profit = value(z.objective)
print("Solution is :",LpStatus[status])
print("The total profit is :",total_profit)
print("Total Profit in Words :",(num2words(total_profit,lang='en_IN')))

Solution is : Optimal
The total profit is : 284063.9700000007
Total Profit in Words : two lakh, eighty-four thousand and sixty-three point nine seven zero zero zero zero zero zero zero seven


## Checking for any disperancy 

In [15]:
for w in range(1,(we+1)):
    for s in stores:
        for p in products:
            if (y[s,p,w].varValue > sales[(s,p,w)]):
                print(y[s,p,w],":",y[s,p,w].varValue,"::",sales[(s,p,w)])


## Repairing the decision variable

In [16]:
for s in stores:
    for p in products:
        for w in week:
            if x[s,p,w].varValue is None:
                x[s,p,w].varValue = 0

# Analyzing our Result

In [17]:
for w in range(1,(we+1)) :
    print("\nCapital Available in week",w,":",lpSum(y[s,p,w-1].varValue*Pprice[p] for s in stores for p in products))
    print("Capital Required in week",w,":",lpSum(sales[s,p,w]*Pcost[p] for s in stores for p in products))
    print("Capital Used in week",w,":",lpSum(x[s,p,w].varValue*Pcost[p] for s in stores for p in products))
    print("Capital Not Used in week",w,":",capital_unused[w].varValue)
    print("Revenue of week for sales",w,":",lpSum(y[s,p,w].varValue*Pprice[p] for s in stores for p in products))
    print("Profit of week",w,":",lpSum(y[s,p,w].varValue*Pprofit[p] for s in stores for p in products))
    print("Opening Inventory Value of week ",w,":",lpSum(inventory[s,p,w].varValue*Pcost[p] for s in stores for p in products))
    print("Inventory value of week after Sales",w,":",lpSum(inventory[s,p,w].varValue*Pcost[p] for s in stores for p in products)-lpSum(y[s,p,w].varValue*Pcost[p] for s in stores for p in products))


Capital Available in week 1 : 0.0
Capital Required in week 1 : 73571.75
Capital Used in week 1 : 0.0
Capital Not Used in week 1 : 0.0
Revenue of week for sales 1 : 76039.68999999992
Profit of week 1 : 20899.52999999998
Opening Inventory Value of week  1 : 299056.0500000006
Inventory value of week after Sales 1 : 243915.89000000068

Capital Available in week 2 : 76039.68999999992
Capital Required in week 2 : 61901.439999999966
Capital Used in week 2 : 76038.52000000005
Capital Not Used in week 2 : 1.17
Revenue of week for sales 2 : 87821.92000000004
Profit of week 2 : 25920.479999999992
Opening Inventory Value of week  2 : 76038.52000000005
Inventory value of week after Sales 2 : 14137.080000000082

Capital Available in week 3 : 87821.92000000004
Capital Required in week 3 : 66469.01000000005
Capital Used in week 3 : 87821.50999999998
Capital Not Used in week 3 : 1.58
Revenue of week for sales 3 : 93753.32999999999
Profit of week 3 : 27284.31999999999
Opening Inventory Value of week  3